In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import matplotlib.pyplot as plt
%matplotlib inline


print('Libraries imported.')

import math

Libraries imported.


In [19]:
# Importing files
property_price=pd.read_csv("https://raw.githubusercontent.com/girishtere/Applied-Data-Science-Capstone/master/Data/pp_latlong.csv")
onehot_venues=pd.read_csv("https://raw.githubusercontent.com/girishtere/Applied-Data-Science-Capstone/master/Data/city_onehot.csv")

In [20]:
#property_price.drop('index',axis=1,inplace=True)
property_price.head()

,index,Locality Name,buy_rate,buy_rate_trim,buy_rate_lower,buy_rate_higher,buy_rate_avg,Buy Rates Q/Q,Rent_1B,Rent_1B_trim,Rent_1B_Lower,Rent_1B_Upper,Rent_1B_avg,Rent_1B_persqf,Rent_2B,Rent_2B_trim,Rent_2B_Lower,Rent_2B_Upper,Rent_2B_avg,Rent_2B_persqf,Rent_3B,Rent_3B_trim,Rent_3B_Lower,Rent_3B_Upper,Rent_3B_avg,Rent_3B_persqf,Latitude,Longitude
0,0,4 Bunglows,"Rs.18,828 - 20,825/sq. ft.","18,828 - 20,825",18828.0,20825.0,19826.5,-1.48%,"Rs.24,225 - 31,025","24,225 - 31,025",24225.0,31025.0,27625.0,139%,"Rs.43,350 - 48,450","43,350 - 48,450",43350.0,48450.0,45900.0,232%,"Rs.53,780 - 59,755","53,780 - 59,755",53780.0,59755.0,56767.5,286%,NaN,NaN
1,1,Aarey Milk Colony,"Rs.7,225 - 7,862/sq. ft.","7,225 - 7,862",7225.0,7862.0,7543.5,4.71%,"Rs.16,371 - 20,464","16,371 - 20,464",16371.0,20464.0,18417.5,244%,"Rs.21,216 - 23,338","21,216 - 23,338",21216.0,23338.0,22277.0,295%,"Rs.23,899 - 29,330","23,899 - 29,330",23899.0,29330.0,26614.5,353%,19.156129,72.870722
2,2,Abhyudaya Nagar,"Rs.26,605 - 28,858/sq. ft.","26,605 - 28,858",26605.0,28858.0,27731.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rs.63,954 - 68,395","63,954 - 68,395",63954.0,68395.0,66174.5,239%,NaN,NaN,NaN,NaN,NaN,NaN,18.990477,72.844057
3,3,Adaigaon,"Rs.4,462 - 4,802/sq. ft.","4,462 - 4,802",4462.0,4802.0,4632.0,7.92%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Adharwadi,"Rs.4,675 - 5,398/sq. ft.","4,675 - 5,398",4675.0,5398.0,5036.5,-7.26%,"Rs.5,142 - 6,078","5,142 - 6,078",5142.0,6078.0,5610.0,111%,"Rs.7,268 - 8,882","7,268 - 8,882",7268.0,8882.0,8075.0,160%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:

lat_long=property_price[['Locality Name','Latitude','Longitude','buy_rate_avg','Rent_1B_avg','Rent_2B_avg','Rent_3B_avg']].dropna()
lat_long.rename(columns={"Locality Name": "Neighborhood"},inplace=True)
lat_long['rent_1_bk']=(lat_long['Rent_1B_avg'].mean(skipna=True)/lat_long['buy_rate_avg'].mean(skipna=True))*lat_long['Rent_1B_avg']
lat_long['rent_2_bk']=(lat_long['Rent_2B_avg'].mean(skipna=True)/lat_long['buy_rate_avg'].mean(skipna=True))*lat_long['Rent_2B_avg']
lat_long['rent_3_bk']=(lat_long['Rent_3B_avg'].mean(skipna=True)/lat_long['buy_rate_avg'].mean(skipna=True))*lat_long['Rent_3B_avg']
lat_long.head()

,Neighborhood,Latitude,Longitude,buy_rate_avg,Rent_1B_avg,Rent_2B_avg,Rent_3B_avg,rent_1_bk,rent_2_bk,rent_3_bk
1,Aarey Milk Colony,19.156129,72.870722,7543.5,18417.5,22277.0,26614.5,23362.288971,48743.903219,93897.994110
6,Airoli,19.158515,72.999402,10561.5,14259.0,22567.5,32742.0,18087.301666,49379.541046,115516.283348
10,Ambedkar Nagar,19.070822,72.828865,16235.0,25239.5,38675.0,56104.0,32015.881226,84624.072226,197939.208385
21,Andheri (East),19.115883,72.854202,16809.0,25478.5,39634.5,54910.0,32319.048706,86723.536927,193726.684950
22,Andheri (West),19.117249,72.833968,21058.5,25712.5,41536.0,66139.0,32615.873770,90884.174893,233343.456855


In [22]:
#onehot_venues.drop('Unnamed: 0',axis=1,inplace=True)
onehot_venues=pd.merge(onehot_venues,lat_long[['Neighborhood','buy_rate_avg','Rent_1B_avg','Rent_2B_avg','Rent_3B_avg']],on='Neighborhood',how='left')
onehot_venues.head()

,Unnamed: 0,Neighborhood,arts_culture_recreation,bars_nightlife,business_hub,café_fastfoods,cuisine_restaurants,education_colleges,kids_family_residential,nature_view,regular_restaurants,shopping,sports_fitness,stores_daily_conveniences,tourist_interest,transport_vicinity,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Mapping,buy_rate_avg,Rent_1B_avg,Rent_2B_avg,Rent_3B_avg
0,0,Aarey Milk Colony,0,0,0,0,0,0,0,0,1,0,0,0,0,0,19.156129,72.870722,Aarey Garden Restaurant,19.155514,72.878236,Indian Restaurant,regular_restaurants,7543.5,18417.5,22277.0,26614.5
1,1,Aarey Milk Colony,0,0,0,0,0,0,0,0,1,0,0,0,0,0,19.156129,72.870722,Panchvati Fast Food Corner,19.157628,72.874506,Fast Food Restaurant,regular_restaurants,7543.5,18417.5,22277.0,26614.5
2,2,Aarey Milk Colony,0,0,0,0,0,0,0,1,0,0,0,0,0,0,19.156129,72.870722,Chota Kashmir Boating,19.160905,72.872482,Lake,nature_view,7543.5,18417.5,22277.0,26614.5
3,3,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,1,0,19.156129,72.870722,Chota Kashmir,19.163542,72.873487,Monument / Landmark,tourist_interest,7543.5,18417.5,22277.0,26614.5
4,4,Abhyudaya Nagar,0,0,0,0,0,0,0,0,1,0,0,0,0,0,18.990477,72.844057,Ladu Samrat,18.994526,72.836574,Indian Restaurant,regular_restaurants,NaN,NaN,NaN,NaN


In [23]:
from math import radians, sin, cos, acos
def calculate_distance(lat1,lon1,lat2,lon2):
    slat = radians(float(lat1))
    slon = radians(float(lon1))
    elat = radians(float(lat2))
    elon = radians(float(lon2))
    dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    return dist

In [24]:
print(calculate_distance(19.156129,72.870722,18.990477,72.844057))

18.631628055232003


In [25]:
#location='Aarey Milk Colony'
#max_distance=5
roomType=['buy_rate_avg','Rent_1B_avg','Rent_2B_avg','Rent_3B_avg']
max_amount=25000
order_type=1
data={'arts_culture_recreation':0,
      'bars_nightlife':0,
      'business_hub':0,
      'café_fastfoods':8,
      'cuisine_restaurants':0,
      'education_colleges':0,
      'kids_family_residential':0,
      'nature_view':0,
      'regular_restaurants':0,
      'shopping':0,
      'sports_fitness':0,
      'stores_daily_conveniences':9,
      'tourist_interest':0,
      'transport_vicinity':10}

user_preference= pd.Series(data).to_frame(name='preference')
user_preference

,preference
arts_culture_recreation,0
bars_nightlife,0
business_hub,0
café_fastfoods,8
cuisine_restaurants,0
education_colleges,0
kids_family_residential,0
nature_view,0
regular_restaurants,0
shopping,0


In [26]:


city_filtered=onehot_venues[onehot_venues[roomType[order_type]]< max_amount]
city_filtered.dropna(inplace=True)




city_grouped=city_filtered.drop('Neighborhood Latitude',1).drop('Neighborhood Longitude',1).drop('Venue',1).drop('Venue Latitude',1).drop('Venue Longitude',1).drop('Venue Category',1).drop('Venue Mapping',1).drop('Rent_3B_avg',1).drop('Rent_2B_avg',1).drop('Rent_1B_avg',1).drop('buy_rate_avg',1)
city_grouped=city_grouped.groupby('Neighborhood').sum().reset_index()
city_grouped.index.name='index'
city_grouped['Neighborhood_id']=city_grouped.index

user_table=city_grouped.drop('Neighborhood',1).drop('Neighborhood_id',1)
for k in range(1,15):
    city_grouped.iloc[:,k]=city_grouped.iloc[:,k]/city_grouped.iloc[:,k].max()

    
user_preference.index.name='venue_type'
user_choice=pd.Series(user_table.columns.tolist()).to_frame(name='venue_type')
user_profile=pd.merge(user_choice,user_preference,on='venue_type',how='left')
user_profile.fillna(0,inplace=True)
user_profile.set_index('venue_type',inplace=True)
user_rating=user_profile.squeeze()
user_rating

recommendation_table=(user_table*user_rating).sum(axis=1)/user_rating.sum()
recommendation_table=recommendation_table.sort_values(ascending=False)

recommendations=city_grouped.loc[city_grouped['Neighborhood_id'].isin(recommendation_table.head(5).keys())]

recommendation_plot=pd.merge(recommendations[['Neighborhood']],lat_long.drop_duplicates(),on='Neighborhood')
recommendation_plot.head()


latitude =recommendation_plot.Latitude.mean()
longitude =recommendation_plot.Longitude.mean()

venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around center

# add coordinates to the map as blue circle markers
for lat, lng, label in zip(recommendation_plot.Latitude, recommendation_plot.Longitude, recommendation_plot['Neighborhood']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.7
        ).add_to(venues_map)

# saving map to html
venues_map.save(outfile='recommendation.html')

print('Done')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


AttributeError: module 'folium.features' has no attribute 'CircleMarker'